In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL-Advanced


# 간단한 예시

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

# 체인을 생성합니다.
chain = (
    prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)

In [4]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "2+2 는 무엇인가요?"})

'수학'

In [5]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [6]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "Google은 어떤 회사인가요?"})

'기타'

In [7]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

# 사용자 정의 함수 사용하기

In [8]:
def route(info):
    # 주제에 "수학"이 포함되어 있는 경우
    if "수학" in info["topic"].lower():
        # datascience_chain을 반환
        return math_chain
    # 주제에 "과학"이 포함되어 있는 경우
    elif "과학" in info["topic"].lower():
        # art_chain을 반환
        return science_chain
    # 그 외의 경우
    else:
        # general_chain을 반환
        return general_chain


In [9]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)

In [10]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 변화와 곡선의 기하학적 성질을 연구하는 데 중점을 두고 있습니다. 미적분학은 크게 미분과 적분으로 나뉘어져 있으며, 미분은 함수의 변화율을 분석하고, 적분은 특정 구간에서의 전체적인 양을 계산하는 데 사용됩니다. \n\n미분을 통해 우리는 함수의 기울기나 속도를 이해할 수 있고, 적분은 면적이나 부피를 구하는 데 유용합니다. 이러한 두 개념은 서로 밀접하게 연결되어 있으며, 기본 정리에 의해 미분과 적분은 서로 상反 보완적인 관계를 가집니다. 미적분은 물리학, 공학, 경제학 등 다양한 분야에서 적용되며, 복잡한 문제를 해결하는 데 필수적인 도구로 자리잡고 있습니다.'

In [11]:
# 과학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "중력은 어떻게 작용하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력은 물체 간의 질량에 비례하고, 그들 사이의 거리의 제곱에 반비례하는 힘이라고 합니다. 즉, 두 물체의 질량이 클수록 중력의 힘도 강해지며, 두 물체 간의 거리가 멀어질수록 중력의 힘은 약해진다는 것입니다. 뉴턴의 법칙에 따르면, 이 힘은 서로를 끌어당기며, 이는 우리 일상에서 사물들이 떨어지거나 행성이 궤도를 도는 원리를 설명합니다. 따라서 중력은 우주에서 물체의 움직임과 상호작용을 결정짓는 중요한 힘임을 알 수 있습니다.'

In [12]:
# 기타 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(Resp Retrieval Augmented Generation)는 정보 검색과 자연어 생성을 결합한 모델로, 질문에 대한 답변 생성 시 외부 데이터베이스에서 관련 정보를 검색하여 활용합니다. 이 방식은 질 높은 응답을 제공하는 데 도움을 줍니다.'

# RunnableBranch

In [13]:
from operator import itemgetter
from langchain_core.runnables import RunnableBranch

In [14]:
branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행합니다.
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행합니다.
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행합니다.
    general_chain,
)
# 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의합니다.
full_chain = (
    {"topic": chain, "question": itemgetter("question")} | branch | StrOutputParser()
)

In [15]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 두 주요 분야인 미분과 적분으로 구성됩니다. 미분은 함수의 변화율을 측정하여 기울기를 구하는 과정이며, 적분은 함수의 면적 또는 전체 값을 구하는 과정입니다. 이 두 개념은 서로 긴밀하게 연결되어 있으며, 미적분학은 물리학, 공학, 경제학 등 다양한 분야에서 많은 응용을 가지고 있습니다. 미적분을 통해 우리는 곡선의 기울기, 면적, 그리고 변화하는 현상을 분석할 수 있습니다.'

In [16]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 물체가 지구 등 특정 천체의 중력장 속에서 얼마나 빠르게 가속되는지를 나타내는 값입니다. 지구의 경우, 일반적으로 중력 가속도는 약 9.81 m/s²로 알려져 있습니다. \n\n중력 가속도를 계산하기 위해서는 중력 법칙을 활용할 수 있습니다. 두 물체간의 중력은 다음과 같은 식으로 표현됩니다:\n\n\\[ F = \\frac{G \\cdot m_1 \\cdot m_2}{r^2} \\]\n\n여기서 \\( F \\)는 두 물체 간의 중력, \\( G \\)는 만유인력 상수, \\( m_1 \\)과 \\( m_2 \\)는 각각의 물체의 질량, \\( r \\)은 두 물체 간의 거리입니다. \n\n지구에서 중력 가속도 \\( g \\)는 질량 \\( m \\)인 물체에 작용하는 중력을 \\( F = m \\cdot g \\)라고 놓고, 이 두 식을 연결하여 아래와 같이 표현할 수 있습니다:\n\n\\[ g = \\frac{G \\cdot M}{r^2} \\]\n\n여기서 \\( M \\)은 지구의 질량, \\( r \\)은 지구의 반지름입니다. 이 식을 이용하여 중력 가속도를 계산할 수 있습니다.'

In [17]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(위치 기반 생성)는 정보 검색과 생성 모델을 결합한 방법으로, 검색된 문서에서 필요한 정보를 추출하여 이를 바탕으로 텍스트를 생성합니다. 이를 통해 더 정확하고 풍부한 응답을 제공할 수 있습니다.'